In [9]:
%reload_ext autoreload
%autoreload 2
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"  # specify which GPU(s) to be used
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm_notebook as tqdm

from inputPipeline_stiching import PandaPatchDataset, crossValDataloader, data_transform
from resnext import Model

In [2]:
data_dir = '../../tile_concat_wy/input/panda-36x256x256-tiles-data-opt/train'
csv_file = '../Train_Data.csv'

In [24]:
sz = 256
bs = 40
tsfm = None
dataset = PandaPatchDataset(csv_file, data_dir, sz, transform=tsfm, N = 36, rand = False)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = bs, shuffle = False, num_workers = 4)
model = Model().cuda()

Using cache found in /home/docker-user/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master


In [25]:
img_features = []
providers = []
for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        img = data['img']
        provider = data['datacenter']
        feat = model(img.cuda().float())['out']
        img_features.append(feat.detach())
        providers += [int(i == 'karolinska') * 2 - 1 for i in provider] ## karolinska:1 radboud:-1

In [26]:
img_features = torch.cat(img_features,0).cpu().numpy()

In [27]:
saved_features = np.concatenate([np.arange(len(providers)).reshape(-1,1),
                           np.asarray(providers).reshape(-1,1), 
                           img_features], axis = 1)

In [29]:
np.save("image_features.npy", saved_features)